<h1><span style="color:red">Generate Aggregate Maps</span></h1>

### This notebook reads numeric and categorical variables from the aim2 survey dataset and lets users compute a cloropleth map of the variable of interest aggregated by zip code, adds it to a new survey version, and publishes the survey to the user's surveys gallery

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [84]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

## 2. Read the survey file and extract numeric and categorical variables

In [ ]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' or '#sortquan' in n}
printmd("<b><span style='color:red'>Numeric variables:</span></b>")
print(var_list)

## 3. Select variable of interest

In [ ]:
# select variable to visualize aggregate zip code data
var_selector = pn.widgets.Select(name='Variable of Interest', options = df.columns.to_list())
selected = var_selector.value
suave_tag = selected.split('#')[1]
pn.Row(var_selector)

## 4. Clean input SuAVE CSV file

In [ ]:
# clean suave zip codes
def clean_zip(data):
    """
    Returns cleaned zip code as float
    """
    if type(data) == float:
        return data
    elif '-' in data:
        return data.split('-')[0]
    return data

df['Zip Code#sortquan'] = df['Zip Code#sortquan'].apply(clean_zip)
df['Zip Code#sortquan'] = df['Zip Code#sortquan'].astype(float)

## 5. Generate aggregate map CSV file

In [ ]:
# generate aggregate data by zip code from variable mapping 
def map_numerical(df, column):
    """
    Returns a dataframe of a numeric variable mean aggregated by zip code
    """
    out = df.groupby('Zip Code#sortquan')[column].mean().to_frame()
    col_selected = column.split('#')
    new_cols = [col_selected[0] + '_mean#' + col_selected[1]]
    out.columns = new_cols
    out = out.reset_index().fillna(0)
    out = out.rename_axis(None, axis=1)
    return out

def map_categorical(df, column):
    """ 
    Returns a dataframe of a categorical variable counts aggregated by zip code
    """
    out = df.groupby('Zip Code#sortquan')[column].value_counts(normalize=True).unstack()
    col_selected = column.split('#')
    new_cols = [col_selected[0] + '_' + i + '#number' for i in out.columns]
    out.columns = new_cols
    out = out.reset_index().fillna(0)
    out = out.rename_axis(None, axis=1)
    return out

# determine variable mapping from suave tag 
if suave_tag == 'number':
    suave_out = map_numerical(df, selected) # mapping for numerical variables
else:
    suave_out = map_categorical(df, selected) # mapping for categorical (radio/checkbox) variables
suave_out

In [ ]:
# add sd zip code geometry data to output
sd_zip = pd.read_csv('sd_zip_boundaries.csv')
sd_zip['ZIP'] = sd_zip['ZIP'].astype(float)
sd_zip['geometry'] = sd_zip['geometry#hiddenmore']
sd_zip = sd_zip[['Community', 'ZIP', 'geometry#hiddenmore']]

In [ ]:
suave_zip = suave_out.merge(sd_zip, left_on = 'Zip Code#sortquan', right_on='ZIP')
suave_zip = suave_zip.drop(columns = 'Zip Code#sortquan')

## 6. Save the new version of CSV file, and give a name to new survey

In [ ]:
new_file = suaveint.save_csv_file(df, absolutePath, csv_file)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

## 7. Generate the survey and create survey URL

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views)